https://adventofcode.com/2019/day/18

In [15]:
import itertools
import collections
import heapq

In [41]:
with open('data/18.txt') as fh:
    data = fh.read()

In [6]:
print(data)

#################################################################################
#.........#...#....h....Q.......#.......#.#...........#...#...#.................#
#.###.###.#.#.###.#############.#.###T#.#.#.#####.###.#.#.#.#.#####.###########.#
#..a#.#.#...#.#...#.#...#.....#.#.#.#.#.#.......#.#...#.#.#.#.....#.#.....#.....#
#####.#.#####.#.###.#.#.#.#.###.#.#.#.###.#######.#.#####.#.#####.#.#.###.#.###.#
#...#.#.......#.#.....#...#.#...#...#...#.#...#...#.#.....#.#.#...#...#...#.#...#
#.#.#.#.#######.#.#########.#.###.#####.###.#.#.###.#.#####.#.#.#######L###.#.###
#.#...#.#.......#.#.......#...#.#.#.....#...#...#.....#.......#.....#.....#.#...#
#.#####.#.#######.#####.#######.#.#.###.#.#.###########.#######.###.#####.#.###.#
#...#.#.#...#.........#...#.....#.#...#.#.#.#...#.......#.....#...#...#...#.#...#
#.#.#.#.###.#########.###.#.#.###.###.#.#.#.#.#.#.#######.###.#######.#.###.#####
#.#.#.#.#...#...#...#...#...#.......#.#.#.#.#.#...#.#.....#.#.......#g#.#.#.....#
###.#.#.#.###.#.

In [7]:
def data2grid(data):
    grid = {}
    for y, line in enumerate(data.split('\n')):
        for x, c in enumerate(line):
            if c != '#': 
                grid[(x,y)] = c
    return grid

In [11]:
grid = data2grid(data)

In [8]:
def nabecoords(pt):
    (x,y) = pt
    return [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]

In [10]:
def reachable_keys(grid, pt, keys):
    q = collections.deque()
    seen = set()
    q.append((pt, 0))
    while q:
        (pt, l) = q.popleft()
        if grid[pt].islower() and grid[pt] not in keys:
            yield l, pt, grid[pt]
            continue
        for nabe in nabecoords(pt):
            if nabe not in grid or nabe in seen:
                continue
            seen.add(nabe)
            if not grid[nabe].isupper() or grid[nabe].lower() in keys:
                q.append((nabe, l+1))

In [12]:
startpos = next(k for k,v in grid.items() if v == '@')
startpos

(40, 40)

In [14]:
list(reachable_keys(grid, startpos, {}))

[(18, (45, 31), 'v'),
 (74, (63, 63), 't'),
 (170, (69, 11), 'g'),
 (186, (7, 77), 'x')]

In [35]:
def keysearch(grid):
    q = []
    seen = set()

    allkeys = frozenset(v for v in grid.values() if v.islower())
    startpos = next(k for k,v in grid.items() if v == '@')
    heapq.heappush(q, (0, startpos, frozenset()))

    while q:
        d, pos, keys = heapq.heappop(q)
        if keys == allkeys:
            return d
        
        if (pos, keys) in seen:
            continue
        seen.add((pos, keys))
    
        for l, nabe, key in reachable_keys(grid, pos, keys):
            heapq.heappush(q, (d + l, nabe, keys | frozenset([key])))

In [38]:
d0 = """\
#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################
"""

In [39]:
g0 = data2grid(d0)

In [40]:
keysearch(g0)

136

In [42]:
%%time
keysearch(data2grid(data))

CPU times: user 2.27 s, sys: 0 ns, total: 2.27 s
Wall time: 2.27 s


3646

In [75]:
grid2 = data2grid(data)

In [56]:
pos = next(k for (k,v) in grid1.items() if v == '@')
pos

(40, 40)

In [57]:
(x,y) = pos
(x,y)

(40, 40)

In [76]:
for (dx, dy) in  [(-1, 0), (0, 0), (1, 0), (0, -1), (0, 1)]:
    (x1, y1) = (x + dx, y + dy)
    if (x1, y1) in grid2:
        del grid2[(x1, y1)]

In [77]:
for (dx, dy) in [(-1, -1), (1, -1), (-1, 1), (1, 1)]:
    grid2[(x+dx, y+dy)] = '@'

In [60]:
for y1 in range(36, 44):
    for x1 in range(36, 44):
        print(grid2.get((x1, y1), ' '), end='')
    print()

     .  
 ... . .
   . .  
...@ @..
        
...@ @..
   . .  
.. . . .


In [61]:
grid2[(39,39)]

'@'

In [62]:
tuple(k for k,v in grid2.items() if v == '@')

((39, 39), (41, 39), (39, 41), (41, 41))

In [79]:
def keysearch2(grid):
    q = []
    allkeys = frozenset(v for v in grid.values() if v.islower())
    startposes = [k for k,v in grid.items() if v == '@']
    seens = [set() for _ in startposes]
    heapq.heappush(q, (0, startposes, frozenset()))

    while q:
        d, poses, keys = heapq.heappop(q)
        if keys == allkeys:
            return d
        
        for i, pos in enumerate(poses):
#             print(i, pos)
            if (pos, keys) in seens[i]:
                continue
            seens[i].add((pos, keys))
    
            for l, nabe, key in reachable_keys(grid, pos, keys):
                npos = poses[:i] + [nabe] + poses[i+1:]
                heapq.heappush(q, (d + l, npos, keys | frozenset([key])))

In [80]:
%%time
keysearch2(grid2)

CPU times: user 915 ms, sys: 0 ns, total: 915 ms
Wall time: 914 ms


1730